In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
from glob import glob

import numpy as np
import pandas as pd
idx = pd.IndexSlice
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import copy

import geopandas as gpd

from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

import gpflow
import tensorflow as tf

code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)

from onoffgpf import OnOffSVGP, OnOffLikelihood

from onoffgpf import OnOffSVGPMC

import pickle

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import contextily as ctx 
import geopandas as gpd 
import os 
from mpl_toolkits.axes_grid1 import make_axes_locatable

2022-12-20 10:09:26.218380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 10:09:26.331018: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-20 10:09:26.336437: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-20 10:09:26.336449: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    https://stackoverflow.com/a/4913653/1748679
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

def fixed_top_X(true_qtr_val,pred_qtr_val, X=10):
    
    top_X_predicted = pred_qtr_val.sort_values(ascending=False)[:X]
    top_X_true = true_qtr_val.sort_values(ascending=False)[:X]


    undisputed_top_predicted = top_X_predicted[top_X_predicted>top_X_predicted.min()]
    num_tied_spots = X - len(undisputed_top_predicted)
    undisputed_top_true = top_X_true[top_X_true > top_X_true.min()]
    num_true_ties = X - len(undisputed_top_true)


    tied_top_predicted = pred_qtr_val[pred_qtr_val==top_X_predicted.min()]
    tied_top_true = true_qtr_val[true_qtr_val==top_X_true.min()]

    error_in_top_true_ties = np.abs(tied_top_true-pred_qtr_val[tied_top_true.index]).sort_values(ascending=True)
    error_in_top_pred_ties = np.abs(true_qtr_val[tied_top_predicted.index]-tied_top_predicted).sort_values(ascending=True)
    top_true_tied_geoids = error_in_top_true_ties[:num_true_ties].index
    top_pred_tied_geoids = error_in_top_pred_ties[:num_tied_spots].index

    best_possible_top_true_geoids = pd.Index.union(undisputed_top_true.index, top_true_tied_geoids)
    best_possible_top_pred_geoids = pd.Index.union(undisputed_top_predicted.index, top_pred_tied_geoids)

    # True values of GEOIDS with highest actual deaths. If ties, finds tied locations that match preds best
    best_possible_true = true_qtr_val[best_possible_top_true_geoids]
    best_possible_pred = true_qtr_val[best_possible_top_pred_geoids]

    assert(len(best_possible_true)==X)
    assert(len(best_possible_pred)==X)

    best_possible_absolute = np.abs(best_possible_true.sum() - best_possible_pred.sum())
    best_possible_ratio = np.abs(best_possible_pred).sum()/np.abs(best_possible_true).sum()


    bootstrapped_tied_indices = np.random.choice(tied_top_predicted.index, (1000, num_tied_spots))
    bootstrapped_all_indices =  [pd.Index.union(undisputed_top_predicted.index,
                                                bootstrap_index) for bootstrap_index in bootstrapped_tied_indices]

    bootstrapped_absolute = np.mean([np.abs(top_X_true.sum() - true_qtr_val[indices].sum()) 
                                     for indices in bootstrapped_all_indices])
    bootstrapped_ratio = np.mean([np.abs(true_qtr_val[indices]).sum()/np.abs(top_X_true).sum()
                                  for indices in bootstrapped_all_indices])
    
    return best_possible_absolute, best_possible_ratio, bootstrapped_absolute, bootstrapped_ratio

In [3]:
data_dir = '/cluster/tufts/hugheslab/datasets/NSF_OD/'
result_dir = os.path.join(data_dir, 'results')
mass_shapefile = os.path.join(data_dir,'shapefiles','MA_2021')



new_tract_shapefile = os.path.join(data_dir,'shapefiles','new_trimmed_tract',
                                   'cb_2020_tract')

mass_gdf = gpd.read_file(new_tract_shapefile)


svi_file = os.path.join(result_dir,'svi_qtr')
svi_gdf = gpd.read_file(svi_file)
svi_gdf = svi_gdf.rename(columns={'INTPTLAT':'lat', 'INTPTLON':'lon', 'GEOID':'grid_squar'})
# Make lat and lon floats
svi_gdf.loc[:, 'lat'] = svi_gdf.lat.astype(float)
svi_gdf.loc[:, 'lon'] = svi_gdf.lon.astype(float)
deaths_gdf = svi_gdf

missing_squares = [square for square in deaths_gdf['grid_squar'].unique() if square not in mass_gdf['GEOID'].unique()]
just_grid = deaths_gdf.loc[(deaths_gdf['year']==2000)&(deaths_gdf['quarter']==4), ['grid_squar','geometry', 'lat', 'lon']]
# Calculate each squares neighbors
neighbors = {}
for _, row in just_grid.iterrows():
    just_grid.loc[:,'haversine'] = just_grid.apply(lambda x: haversine(row['lon'], row['lat'],
                                                                         x['lon'], x['lat']),
                                                  axis=1)
    matching_neighbors = just_grid[just_grid['haversine']<8]['grid_squar'].values
    neighbors[row['grid_squar']]=matching_neighbors
    
tracts = deaths_gdf['grid_squar'].unique()
min_year = deaths_gdf.year.min()
max_year = deaths_gdf.year.max()
deaths_gdf = deaths_gdf.set_index(['grid_squar','year','quarter']).sort_index()

deaths_gdf.loc[idx[:,:,:],'self_t-1'] = deaths_gdf.loc[idx[:,:,:],'deaths'].shift(1, fill_value=0)

for tract in tracts:
     deaths_gdf.loc[idx[tract,:,:],'neighbor_t-1']= \
        deaths_gdf.loc[idx[neighbors[tract],:,:],'self_t-1'].groupby(level=['year', 'quarter']).mean().shift(1, fill_value=0).values
        
timestep = 0

for year in range(min_year, max_year+1):
    for quarter in range(1, 5):
        deaths_gdf.loc[idx[:,year, quarter], 'timestep'] = timestep
        timestep+=1
        
deaths_gdf_with_autoregressive = deaths_gdf.reset_index()

train_x_through_2018 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year']<=2018][['grid_squar','lat', 'lon', 'timestep', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile', 'neighbor_t-1', 'self_t-1']]
train_y_through_2018 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year']<=2018][['grid_squar',  'timestep', 'deaths']]
train_x_through_2019 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year']<=2019][['grid_squar','lat', 'lon', 'timestep', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile', 'neighbor_t-1', 'self_t-1']]
train_y_through_2019 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year']<=2019][['grid_squar', 'timestep', 'deaths']]

x_just_2019 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year']==2019][['grid_squar','lat', 'lon', 'timestep', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile','neighbor_t-1', 'self_t-1']]
y_just_2019 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year']==2019][['grid_squar','timestep', 'deaths']]
x_just_2020 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year']==2020][['grid_squar','lat', 'lon', 'timestep', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile', 'neighbor_t-1', 'self_t-1']]
y_just_2020 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year']==2020][['grid_squar', 'timestep', 'deaths']]

x_just_2019q1 = deaths_gdf_with_autoregressive[(deaths_gdf_with_autoregressive['year']==2019) & (deaths_gdf_with_autoregressive['quarter']==1)][['grid_squar','lat', 'lon', 'timestep', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile','neighbor_t-1', 'self_t-1']]
y_just_2019q1 = deaths_gdf_with_autoregressive[(deaths_gdf_with_autoregressive['year']==2019) & (deaths_gdf_with_autoregressive['quarter']==1)][['grid_squar','timestep', 'deaths']]


In [4]:
spatial_kernel = gpflow.kernels.RBF(2, active_dims=[0,1])
temporal_kernel = gpflow.kernels.RBF(1, active_dims=[2])
demo_kernel =  gpflow.kernels.RBF(7,active_dims=[3,4,5,6,7,8,9])
gaussian_kernel = spatial_kernel + temporal_kernel + demo_kernel

f_kernel = copy.deepcopy(gaussian_kernel)
g_kernel = copy.deepcopy(gaussian_kernel)


2022-12-20 10:11:13.237383: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-20 10:11:13.237423: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: p1cmp078.pax.tufts.edu
2022-12-20 10:11:13.237430: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: p1cmp078.pax.tufts.edu
2022-12-20 10:11:13.237590: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2022-12-20 10:11:13.237619: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.47.3
2022-12-20 10:11:13.237624: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.47.3
2022-12-20 10:11:13.238188: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the fo

In [5]:
likelihood = OnOffLikelihood()
random = np.random.default_rng(seed=1)

In [6]:
M = 100
N = len(train_x_through_2018)
Z = random.choice(train_x_through_2018[['lat','lon', 'timestep', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile', 'neighbor_t-1', 'self_t-1']].values, size=M, replace=False)

Zf = copy.deepcopy(Z)
Zg = copy.deepcopy(Z)

train_dataset = tf.data.Dataset.from_tensor_slices((train_x_through_2018.loc[:, ['lat','lon', 'timestep', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile', 'neighbor_t-1', 'self_t-1']], 
                                                   train_y_through_2018.loc[:,'deaths'].values.reshape(-1,1))).repeat().shuffle(N)


In [7]:
def normcdf(x):
            return 0.5 * (1.0 + tf.math.erf(x / np.sqrt(2.0))) * (1. - 2.e-3) + 1.e-3

def run_adam(model, iterations, out_dir, death_df, learning_rate=0.005):
    """
    Utility function running the Adam optimizer

    :param model: GPflow model
    :param interations: number of iterations
    """
    # Create an Adam Optimizer action
    logf = []
    stat_logs = pd.DataFrame()
    stat_path = os.path.join(out_dir, 'stats.csv')
    model_path = os.path.join(out_dir, 'model.mod')
    training_loss = model.training_loss_closure(compile=True)
    optimizer = tf.optimizers.Adam(learning_rate=learning_rate)#gpflow.optimizers.Adam(learning_rate=0.01)

    @tf.function
    def optimization_step():
        optimizer.minimize(training_loss, model.trainable_variables)

    for step in range(iterations):
        optimization_step()
        if step % 10 == 0:
            elbo = -training_loss().numpy()
            logf.append(elbo)

            maes = []
            xtop = []
            for quarter in range(1, 4 + 1):
                test_x = death_df[(death_df['year'] == 2019) & (
                            death_df['quarter'] == quarter)][
                    ['grid_squar', 'lat', 'lon', 'timestep', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                     'svi_pctile', 'neighbor_t-1', 'self_t-1']]
                test_y = death_df[(death_df['year'] == 2019) & (
                            death_df['quarter'] == quarter)][['grid_squar', 'timestep', 'deaths']]
                _, _, _, fmean, fvar, gmean, gvar, _, _ = model.build_predict(test_x.loc[:,
                                                                          ['lat', 'lon', 'timestep', 'theme_1_pc',
                                                                           'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                                                                           'svi_pctile', 'neighbor_t-1',
                                                                           'self_t-1']].values)
                sg_2019 = tf.math.softplus(fmean * normcdf(gmean) + 2).numpy()
                pred_2019_df = pd.Series(sg_2019.squeeze(), index=test_y.grid_squar)

                maes.append(mean_absolute_error(test_y.deaths, pred_2019_df))
                xtop.append(fixed_top_X(test_y.set_index('grid_squar')['deaths'], pred_2019_df, 100))

            stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
                                         ignore_index=True)

            stat_logs.to_csv(stat_path)
            model.savemodel(model_path)


    return logf

In [8]:
m = OnOffSVGPMC(train_x_through_2018.loc[:, ['lat','lon','timestep','theme_1_pc', 'theme_2_pc',
                                           'theme_3_pc', 'theme_4_pc', 'svi_pctile',
                                           'neighbor_t-1', 'self_t-1']].values,
              train_y_through_2018.loc[:,'deaths'].values.reshape(-1,1)
              ,kernf=f_kernel,
              kerng=g_kernel
              ,likelihood = OnOffLikelihood()
              ,Zf = Zf,
              Zg = Zg, samples=10
             )


In [9]:
outdir='/cluster/home/kheuto01/test_MC_normal/'
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [10]:
logf = run_adam(m, 2000,outdir, deaths_gdf_with_autoregressive, learning_rate=0.05)

/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated

/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated

/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated

/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated

/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated

/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated

/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated

/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stat_logs = stat_logs.append({'iter': step,'elbo':elbo, 'mae': np.mean(maes), 'bpr_100': np.mean([thing[3] for thing in xtop])},
/tmp/ipykernel_179593/429303792.py:49: FutureWarning: The frame.append method is deprecated

In [11]:
m.likelihood.variance

<Parameter: name=chain_of_shift_of_softplus, dtype=float64, shape=[], fn="chain_of_shift_of_softplus", numpy=0.14250082689007684>